In [1]:
#%pip install openai -q

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import ast

from utils.claude_label_utils import predict

from utils.instructions import label_instructions_v515
MODEL_ID = "us.anthropic.claude-sonnet-4-20250514-v1:0"
SYSTEM_PROMPT = label_instructions_v515

In [2]:
df = pd.read_csv("../experiments_501/data/output_dataset.csv")
df.head()

,citing_cluster_id,citing_decision_name,citing_url,citing_opinions,citing_filenames,cited_cluster_id,cited_decision_name,cited_url,cited_name_short,cited_name,cited_name_full,cited_citations,overruled,note,use_full_opinion,filename
0,91306,"Morgan v. United States,113 U.S. 476 (1885)",https://www.courtlistener.com/opinion/91306/mo...,"[{'opinion_id': 91306, 'opinion_api': 'https:/...",['91306_010combined.txt'],88061,Texas v. White (1869),https://www.courtlistener.com/opinion/88061/te...,White,Texas v. White,Texas v. White Et Al.,"['74 U.S. 700', '19 L. Ed. 227', '7 Wall. 700'...",yes,NaN,0,0001.91306_cites_88061.txt
1,91306,"Morgan v. United States,113 U.S. 476 (1885)",https://www.courtlistener.com/opinion/91306/mo...,"[{'opinion_id': 91306, 'opinion_api': 'https:/...",['91306_010combined.txt'],88994,Vermilye & Co. v. Adams Express Co. (1875),https://www.courtlistener.com/opinion/88994/ve...,NaN,Vermilye & Co. v. Adams Express Co.,Vermilye & Co. v. Adams Express Company,"['88 U.S. 138', '22 L. Ed. 609', '21 Wall. 138...",no,NaN,0,0002.91306_cites_88994.txt
2,91306,"Morgan v. United States,113 U.S. 476 (1885)",https://www.courtlistener.com/opinion/91306/mo...,"[{'opinion_id': 91306, 'opinion_api': 'https:/...",['91306_010combined.txt'],87633,Murray v. Lardner (1865),https://www.courtlistener.com/opinion/87633/mu...,Murray,Murray v. Lardner,Murray v. Lardner,"['69 U.S. 110', '17 L. Ed. 857', '2 Wall. 110'...",no,NaN,0,0003.91306_cites_87633.txt
3,91306,"Morgan v. United States,113 U.S. 476 (1885)",https://www.courtlistener.com/opinion/91306/mo...,"[{'opinion_id': 91306, 'opinion_api': 'https:/...",['91306_010combined.txt'],88240,Texas v. Hardenberg (1869),https://www.courtlistener.com/opinion/88240/te...,Hardenberg,Texas v. Hardenberg,Texas v. Hardenberg,"['77 U.S. 68', '19 L. Ed. 839', '10 Wall. 68',...",no,NaN,0,0004.91306_cites_88240.txt
4,91306,"Morgan v. United States,113 U.S. 476 (1885)",https://www.courtlistener.com/opinion/91306/mo...,"[{'opinion_id': 91306, 'opinion_api': 'https:/...",['91306_010combined.txt'],88693,Huntington v. Texas (1873),https://www.courtlistener.com/opinion/88693/hu...,Huntington,Huntington v. Texas,Huntington v. Texas; Texas v. Huntington,"['83 U.S. 402', '21 L. Ed. 316', '16 Wall. 402...",no,NaN,0,0005.91306_cites_88693.txt


In [3]:
citing_dict = df[['citing_cluster_id', 'citing_decision_name', 'citing_opinions', 'citing_filenames']].drop_duplicates().set_index('citing_cluster_id')[['citing_decision_name', 'citing_filenames']].to_dict(orient='index')
len(citing_dict.keys())

133

In [4]:
cited_dict = {}

citations_df = df[['citing_cluster_id', 'cited_cluster_id',
       'cited_decision_name', 'cited_name_short', 'cited_name',
       'cited_name_full', 'cited_citations']]

for _, row in citations_df.iterrows():
    citing = row['citing_cluster_id']
    cited = row['cited_cluster_id']
    
    # Extract metadata (everything except citing and cited IDs)
    metadata = row.drop(labels=['citing_cluster_id', 'cited_cluster_id']).to_dict()
    
    if citing not in cited_dict:
        cited_dict[citing] = {}
    
    cited_dict[citing][cited] = metadata
    
len(cited_dict.keys())

133

In [5]:
%%time

file_path = "../experiments_501/data/raw_citing_opinions"
raw_results_df, parsed_results_df = predict(citing_dict, cited_dict, model_id=MODEL_ID, system_prompt=SYSTEM_PROMPT, data_folder=file_path)

INFO:botocore.tokens:Loading cached SSO token for flp-sso
INFO:botocore.tokens:Loading cached SSO token for flp-sso
INFO:root:Processed 1: citing_id 92291
INFO:root:Processed 2: citing_id 93904
INFO:root:Processed 3: citing_id 93311
INFO:root:Processed 4: citing_id 98124
INFO:root:Processed 5: citing_id 91306
INFO:root:Processed 6: citing_id 92059
INFO:root:Processed 7: citing_id 99901
INFO:root:Processed 8: citing_id 99004
INFO:root:Processed 9: citing_id 101368
ERROR:root:JSON parsing error: Expecting ',' delimiter: line 1 column 2153 (char 2152). Response json str: {
  "97893": {
    "cited_decision": "Bauer & Cie v. O'Donnell",
    "treatment": "Mentioned",
    "quote": "But this court, when the question came before it in Bauer v. O'Donnell, 229 U. S. 1, rejecting plausible argument and. adhering to the language of the statute from which all patent right is derived, refused to give such a construction to the act of Congress, and decided that the owner of a patent is not authorized 

CPU times: user 11.9 s, sys: 1.38 s, total: 13.3 s
Wall time: 24min 41s


In [6]:
raw_results_df

,citing_cluster_id,model,input_tokens,output_tokens,raw_results
0,92291,us.anthropic.claude-sonnet-4-20250514-v1:0,7216,687,"```json\n{\n ""88699"": {\n ""cited_decision""..."
1,93904,us.anthropic.claude-sonnet-4-20250514-v1:0,5734,1114,"```json\n{\n ""93354"": {\n ""cited_decision""..."
2,93311,us.anthropic.claude-sonnet-4-20250514-v1:0,15475,808,"```json\n{\n ""92988"": {\n ""cited_decision""..."
3,98124,us.anthropic.claude-sonnet-4-20250514-v1:0,4502,353,"```json\n{\n ""96356"": {\n ""cited_decision""..."
4,91306,us.anthropic.claude-sonnet-4-20250514-v1:0,11941,1333,"```json\n{\n ""88061"": {\n ""cited_decision""..."
...,...,...,...,...,...
128,111940,,,,
129,118317,us.anthropic.claude-sonnet-4-20250514-v1:0,27374,1496,"```json\n{\n ""118011"": {\n ""cited_decision..."
130,118386,us.anthropic.claude-sonnet-4-20250514-v1:0,61938,1699,"```json\n{\n ""118133"": {\n ""cited_decision..."
131,118273,us.anthropic.claude-sonnet-4-20250514-v1:0,28145,1500,"```json\n{\n ""111516"": {\n ""cited_decision..."


In [7]:
parsed_results_df = parsed_results_df.rename(columns={'treatment': 'prediction'})
parsed_results_df

,citing_cluster_id,cited_cluster_id,cited_decision,prediction,quote,rationale
0,92291,88699,Osborne v. Mobile,Overruled,In view of the course of decisions which have ...,The Acting Decision explicitly states that the...
1,92291,89664,Pensacola Telegraph Co. v. Western Union Teleg...,Mentioned,In the case of The Pensacola Telegraph Company...,The Acting Decision cites this case favorably ...
2,92291,90600,Telegraph Co. v. Texas,Mentioned,"In Western Union Telegraph Company v. Texas, 1...",The Acting Decision references this case posit...
3,92291,91869,Robbins v. Shelby County Taxing District,Mentioned,It comes plainly within the principle of the d...,The Acting Decision cites this case as support...
4,92291,91985,Philadelphia & Southern Steamship Co. v. Penns...,Mentioned,It comes plainly within the principle of the d...,The Acting Decision cites this case alongside ...
...,...,...,...,...,...,...
930,118273,105018,Youngstown Sheet & Tube Co. v. Sawyer,Mentioned,The Court of Appeals began its analysis of thi...,The Court references Youngstown as establishin...
931,118273,98773,New York Ex Rel. Kennedy v. Becker,Mentioned,"In New York ex rel. Kennedy v. (1916), the Cou...",The Court references Kennedy in Justice Thomas...
932,118273,103642,Tulee v. Washington,Mentioned,The conservation necessity standard appears to...,The Court references Tulee in Justice Thomas's...
933,118273,96748,Winters v. United States,Mentioned,We have held that Indian treaties are to be in...,The Court cites Winters as supporting authorit...


In [8]:
raw_results_df["input_tokens"].replace('', 0).astype(int).sum()

/var/folders/rx/3t2jtk8j69j_db_cydq_v6wc0000gn/T/ipykernel_31438/1591752532.py:1: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  raw_results_df["input_tokens"].replace('', 0).astype(int).sum()


np.int64(2157754)

In [9]:
raw_results_df["output_tokens"].replace('', 0).astype(int).sum()

/var/folders/rx/3t2jtk8j69j_db_cydq_v6wc0000gn/T/ipykernel_31438/2426617724.py:1: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  raw_results_df["output_tokens"].replace('', 0).astype(int).sum()


np.int64(151457)

In [10]:
raw_results_df.to_csv("predictions/claude4_raw_results.csv", index=False)
parsed_results_df.to_csv("predictions/claude4_parsed_results_df.csv", index=False)

In [11]:
redo_ids = parsed_results_df[parsed_results_df["prediction"] == ""]["citing_cluster_id"].to_list()

In [12]:
redo = df[df["citing_cluster_id"].isin(redo_ids)]
len(redo)

158

In [13]:
citing_dict = redo[['citing_cluster_id', 'citing_decision_name', 'citing_opinions', 'citing_filenames']].drop_duplicates().set_index('citing_cluster_id')[['citing_decision_name', 'citing_filenames']].to_dict(orient='index')
len(citing_dict.keys())

8

In [14]:
cited_dict = {}

citations_df = redo[['citing_cluster_id', 'cited_cluster_id',
       'cited_decision_name', 'cited_name_short', 'cited_name',
       'cited_name_full', 'cited_citations']]

for _, row in citations_df.iterrows():
    citing = row['citing_cluster_id']
    cited = row['cited_cluster_id']
    
    # Extract metadata (everything except citing and cited IDs)
    metadata = row.drop(labels=['citing_cluster_id', 'cited_cluster_id']).to_dict()
    
    if citing not in cited_dict:
        cited_dict[citing] = {}
    
    cited_dict[citing][cited] = metadata
    
len(cited_dict.keys())

8

In [15]:
%%time

file_path = "../experiments_501/data/raw_citing_opinions"
redo_raw_results_df, redo_parsed_results_df = predict(citing_dict, cited_dict, model_id=MODEL_ID, system_prompt=SYSTEM_PROMPT, data_folder=file_path)

INFO:root:Processed 1: citing_id 107252
INFO:root:Processed 2: citing_id 108990
INFO:root:Processed 3: citing_id 103870


KeyboardInterrupt: 

In [22]:
redo_raw_results_df

,citing_cluster_id,model,input_tokens,output_tokens,raw_results
0,101913,,,,
1,111308,anthropic.claude-3-5-sonnet-20241022-v2:0,31469,1302,"{\n ""109499"": {\n ""cited_decision"": ""Natio..."
2,109532,,,,
3,104610,anthropic.claude-3-5-sonnet-20241022-v2:0,10492,2220,"{\n ""103522"": {\n ""cited_decision"": ""Olsen..."
4,109881,anthropic.claude-3-5-sonnet-20241022-v2:0,33474,1631,"{\n ""106170"": {\n ""cited_decision"": ""Monro..."
5,111555,anthropic.claude-3-5-sonnet-20241022-v2:0,12276,1216,"{\n ""110478"": {\n ""cited_decision"": ""Parra..."
6,112640,,,,
7,118023,anthropic.claude-3-5-sonnet-20241022-v2:0,26534,1466,"{\n ""110312"": {\n ""cited_decision"": ""Centr..."
8,118011,anthropic.claude-3-5-sonnet-20241022-v2:0,109952,1469,"{\n ""92702"": {\n ""cited_decision"": ""Hans v..."
9,118386,,,,


In [23]:
redo_parsed_results_df = redo_parsed_results_df.rename(columns={'treatment': 'prediction'})
redo_parsed_results_df

,citing_cluster_id,cited_cluster_id,cited_decision,prediction,quote,rationale
0,101913,,,,,
1,111308,109499,National League of Cities v. Usery,Overruled,"National League of Cities v. Usery, 426 U. S. ...","The Court explicitly overrules this decision, ..."
2,111308,110881,Equal Employment Opportunity Commission v. Wyo...,Mentioned,"Of course, we continue to recognize that the S...",The Court cites this case while discussing the...
3,111308,110516,Hodel v. Virginia Surface Mining & Reclamation...,Mentioned,The prerequisites for governmental immunity un...,The Court references Hodel to explain the prev...
4,111308,110673,United Transportation Union v. Long Island Rai...,Mentioned,"During the pendency of those appeals, Transpor...",The Court references this case as part of the ...
...,...,...,...,...,...,...
76,118011,109520,Fitzpatrick v. Bitzer,Distinguished,"Previously, in conducting that inquiry, we hav...",The Court distinguishes Fitzpatrick as dealing...
77,118011,111949,Welch v. Texas Department of Highways & Public...,Mentioned,"In the past, we have assumed that a plain-stat...",The Court cites Welch as precedent for the pla...
78,118011,112800,Puerto Rico Aqueduct and Sewer Authority v. Me...,Mentioned,See also Puerto Rico Aqueduct and Sewer Author...,The Court cites Puerto Rico Aqueduct to suppor...
79,118011,658639,Seminole Tribe of Florida v. State of Florida,Mentioned,The Court of Appeals for the Eleventh Circuit ...,The Court merely describes the procedural hist...


In [27]:
raw_results_df = raw_results_df[~raw_results_df["citing_cluster_id"].isin(redo_ids)]
raw_results_df = pd.concat([raw_results_df, redo_raw_results_df])

In [28]:
raw_results_df[raw_results_df["citing_cluster_id"].isin(redo_ids)]

,citing_cluster_id,model,input_tokens,output_tokens,raw_results
0,101913,,,,
1,111308,anthropic.claude-3-5-sonnet-20241022-v2:0,31469,1302,"{\n ""109499"": {\n ""cited_decision"": ""Natio..."
2,109532,,,,
3,104610,anthropic.claude-3-5-sonnet-20241022-v2:0,10492,2220,"{\n ""103522"": {\n ""cited_decision"": ""Olsen..."
4,109881,anthropic.claude-3-5-sonnet-20241022-v2:0,33474,1631,"{\n ""106170"": {\n ""cited_decision"": ""Monro..."
5,111555,anthropic.claude-3-5-sonnet-20241022-v2:0,12276,1216,"{\n ""110478"": {\n ""cited_decision"": ""Parra..."
6,112640,,,,
7,118023,anthropic.claude-3-5-sonnet-20241022-v2:0,26534,1466,"{\n ""110312"": {\n ""cited_decision"": ""Centr..."
8,118011,anthropic.claude-3-5-sonnet-20241022-v2:0,109952,1469,"{\n ""92702"": {\n ""cited_decision"": ""Hans v..."
9,118386,,,,


In [29]:
parsed_results_df = parsed_results_df[~parsed_results_df["citing_cluster_id"].isin(redo_ids)]
parsed_results_df = pd.concat([parsed_results_df, redo_parsed_results_df])

In [30]:
parsed_results_df["prediction"].value_counts()

prediction
Mentioned        641
Overruled        154
Distinguished    139
Limited           41
Criticized        28
Reversed          18
                   4
Questioned         2
Name: count, dtype: int64

In [31]:
raw_results_df["citing_cluster_id"].nunique()

133

In [32]:
parsed_results_df["citing_cluster_id"].nunique()

133

In [33]:
errored_citing_ids = set(raw_results_df["citing_cluster_id"].unique()) - set(parsed_results_df["citing_cluster_id"].unique())
errored_citing_ids

set()

In [16]:
prediction_df = parsed_results_df

In [17]:
set(df["citing_cluster_id"].unique()) - set(prediction_df["citing_cluster_id"].unique())

{np.int64(98917)}

In [18]:
prediction_df.to_csv("predictions/claude4_prediction_df.csv", index=False)